## A quick recognition of our training text

In [3]:
#read it to inspect our training text
with open('tinyShakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

# let's look at the first 1000 characters
print(text[:1000])

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
print()

length of dataset in characters:  1115393
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

## Encoder

In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
# let's now encode the entire text dataset and store it into a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [6]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
train_data[:block_size+1]

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} target is {target}")

when input is tensor([18]) target is 47
when input is tensor([18, 47]) target is 56
when input is tensor([18, 47, 56]) target is 57
when input is tensor([18, 47, 56, 57]) target is 58
when input is tensor([18, 47, 56, 57, 58]) target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [8]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for prediction?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # 生成 batch_size 个随机起始索引, len(data) - block_size 确保 x 取的范围不会超出数据长度
    x = torch.stack([data[i:i+block_size] for i in ix]) # shape of x: (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # shape of y: (batch_size, block_size)
    return x, y 

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('target:')
print(yb.shape)
print(yb)

print('------------------------------------------------------')

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
target:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
------------------------------------------------------
when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 

In [9]:
print(xb) # our input to the transformer

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])


## A simple Bigram model

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

# 每次仅仅根据前一个token来预测下一个token的内容
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads of logits for next token in a lookup tabel
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        '''
        在 PyTorch 中, nn.Embedding(d1, d2) 创建了一个查找表 (lookup table), 用于将离散索引 (token ID) 映射到高维向量。
        其中包含 d1 个向量，每个向量的维度是 d2. 这些向量的初始值是随机的, 并且符合标准正态分布的一种变体。
        '''
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C), where C is vocab_size = 65

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # F.log_softmax(logits, dim=-1)：对 logits 进行 softmax 归一化，并取对数

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions, where idx is (B, C) dimension
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # 按照 probs 的概率大小，随机选取 num_samples 个样本, shape: (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
# 当预测完全随机时 loss = log(1/65)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [11]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

# train the biagram model
batch_size = 32
for steps in range(10000):

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True) # 清空梯度，防止梯度累积
  loss.backward()
  optimizer.step()

print(loss.item())

2.4699912071228027


In [12]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henouratucenonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h haybet--s n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scat t tridesar, wnl'shenou


## The mathematiacal trick in self-attention

In [13]:
# consider the fllowing toy example
torch.manual_seed(1337)
B, T, C = 4, 8 ,2 # batch, time channels
x = torch.randn(B, T, C)
print(x.shape)
print(x)

torch.Size([4, 8, 2])
tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [14]:
# We want x[b,t] = mean_{i<=t} x[b,i]
# version 1
xbow = torch.zeros(B, T, C)
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] #(t,C)
    xbow[b,t] = torch.mean(xprev, 0)

print(x[0])
print(xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [15]:
# 前面的代码实现了沿着某个维度对张量求平均的功能, 虽然满足要求但是效率较低
# 下面使用矩阵乘法来高效实现该功能
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [16]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

print(xbow[0])
print(xbow2[0])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [18]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)
wei = F.softmax(wei, dim=-1)
print(wei)
xbow3 = wei @ x

print(xbow[0])
print(xbow3[0])

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[ 0.18

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channel
x = torch.randn(B, T, C)

# Let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
out = wei @ v # (B, T, T) @ (B, T, 16) ---> (B, T, 16)

print(wei)
print(out.shape)

tensor([[[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
         [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
         [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],

        [[0.0443, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0042, 0.0375, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0560, 0.0210, 0.2496, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3679, 0.1441, 0.4929, 0.0438, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0088, 0.1052, 0.0604, 0.5847, 0.2046, 0.0000, 0.0000, 0.0000],
         [0.0367, 0.089

## LayerNorm

In [22]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

## Decoder-only GPT model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu' # to run on a GPU if you have it
eval_iters = 200
n_embd = 384 # number of embedding dimension
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('tinyShakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars) # vocab_size = 65
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # 生成 batch_size 个随机起始索引, len(data) - block_size 确保 x 取的范围不会超出数据长度
    x = torch.stack([data[i:i+block_size] for i in ix]) # shape of x: (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # shape of y: (batch_size, block_size)
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y) # foward 函数除了能前向传播输出结果, 还能够计算并返回 loss
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# one head of self-attention
class Head(nn.Module):
    """ one head of self-attention """

    # self-attention 的核心思想是用 key 和 query 的内积来表征不同 token 之间的"匹配度",
    # 再以这个"匹配度"为权重计算 value 的加权和作为 self-attention 的最终输出结果。
    def __init__(self, head_size):
        super().__init__()

        # n_embd=384, head_size=64, num_heads=6
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # (T, T) 下三角阵
        '''
        在 torch.nn.Module 中, register_buffer 允许你注册不作为参数更新的变量(即不参与梯度计算),
        但仍会被存储在模型的 state_dict 中，并随模型一起移动到 GPU/CPU。
        '''

        self.dropout = nn.Dropout(dropout) # 创建一个 Dropout 层, 在训练时随机将部分权重置为零。作用是防止过拟合, 提高模型泛化能力。

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        '''
        k.shape[-1] 就是 head_size, 当 head_size 维度很大时, q @ k^T 的值可能很大,
        为了防止 softmax 输出过于极端, 在 Transformer 论文中, 注意力分数会进行缩放
        '''

        # 通过下面的语句将 wei 转化为下三角矩阵, 实现 masked self-attention 功能
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei) 
        '''
        dropout=0.2, 在训练时每个元素有 20% 概率被置为 0, 其余 80% 的值会放大 1/0.8=1.25 倍, 以保持期望值不变。
        这样做的目的是：让模型在不同的训练阶段不会过度依赖某些特定的权重，而是学会利用所有的输入信息。
        在测试(推理)阶段, Dropout 层将不会再起作用, 权重不会被随机置零。
        '''

        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) 
        # 在 MultiHeadAttention 结构中, 需要 self.proj 这样一个全连接层, 
        # 其作用是对多个 head 计算出来的 attention 结果进行整合并映射回输入的维度。
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # x: (B,T,C) [B=batch_size=64, T=block_size=256, C=n_embd=384]
        # h(x): (B,T,head_size) [B=batch_size=64, T=block_size=256, head_size=64]
        # out: (B,T,C) [B=batch_size=64, T=block_size=256, C=n_embd=384]
        out = self.dropout(self.proj(out))
        return out

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head  # n-embd=384, n_head=6, head_size=64
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Residual 层（残差连接） 是一种跳跃连接（skip connection），它的作用是缓解梯度消失问题、加速训练、稳定深层网络的优化
        # 每个Block的构成：layer norm + self-attention + residual layer + layer norm + feed foward layer + residual layer
        # 注：Transforemer Block 中的标准化均为 layer normalization 而非 batch normalization

        # x: (B,T,C) [B=batch_size=64, T=block_size=256, C=n_embd=384]
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        '''
        *[...]表示解包 (unpacking), 用于把列表中的 Block 传递给 nn.Sequential, 相当于:
        self.blocks = nn.Sequential(Block(...), Block(...), ..., Block(...))
        这样就形成了一个 层叠的 Transformer 模型，即输入 x 会依次通过 n_layer=6 个 Block
        '''
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm 
        '''
        LayerNorm 是针对单个样本的 C 维度进行归一化;
        BatchNorm 是针对整个 batch 在 B 维度进行归一化
        '''
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C) [B=batch_size=64, T=block_size=256, C=n_embd=384]
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C) 这里在运算时会先将 pos_emb broadcast 为 (B, T, C) 的 Tensor
        x = self.blocks(x) # (B,T,C) 一般情况下 self-attention block 的输入输出张量有着相同的规格
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # 在自回归 (auto regressive) 模型中, cross entropy loss = MLE loss

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # # 按照 probs 的概率大小，随机选取 num_samples 个样本, shape: (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device) # for using GPU

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# training loop
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

'''
我们计算该模型的总参数个数，并给出计算公式
1.计算 Token Embedding 层
  模型使用 nn.Embedding(vocab_size, n_embd) 进行词嵌入:
  参数个数=vocab_size*n_embd=65*384=24960

2.计算 Position Embedding 层
  模型使用 nn.Embedding(block_size, n_embd) 进行位置嵌入:
  参数个数=block_size*n_embd=256*384=98304

3.计算 Transformer Block(共 n_layer=6 层)
  每个 Block 由 Multi-Head Attention 和 FeedForward 组成。

  3.1 Multi-Head Attention
      Multi-Head Attention 由 n_head=6 个 Head 组成,
      每个 Head 具有 head_size = n_embd // n_head = 384 // 6 = 64 维度。
      每个 Head 包含: Key (K), Query (Q), Value (V) 这三个线性变换(矩阵):
      每个线性变换的参数个数=3*(n_embd*head_size)=3*(384*64)=73728, 
      所有 Head 总计：参数个数=6*73728=442368
      Multi-Head Attention 的最终线性层(将拼接后的多头表示映射回 n_embd 维度)的参数个数:
      参数个数=n_embd*n_embd=384*384=147456
      Multi-Head Attention 总参数: 442368+147456=589824

  3.2 FeedForward 层
      包含两层全连接：
      第一层: 输入维度=n_embd, 输出维度=4*n_embd
      参数个数=n_embd*(4*n_embd)=384*1536=589824
      第二层: 输入维度=4*n_embd,输出维度=n_embd
      参数个数=(4*n_embd)*n_embd=1536*384=589824
      FeedForward 总参数:589824+589824=1179648

  3.3 LayerNorm
      LayerNorm 有两个(不可训练)的参数: γ和β, 每个大小为 n_embd,
      参数个数=2*n_embd=2*384=768
      每个 Block 有 2 个 LayerNorm:
      参数个数=768*2=1536

  参数个数=589824+1179648+1536=1771008
  6 层 Transformer的总参数:1771008*6=10626048

4.计算最终 LayerNorm
  模型的最终LayerNorm:参数个数=2*n_embd=768

5.计算输出投影层
  输出层 lm_head = nn.Linear(n_embd, vocab_size):
  参数个数=n_embd*vocab_size=384*65=24960

总参数个数=24960+98304+10626048+768+24960=10775040≈10.78M
'''


10.788929 M parameters
step 0: train loss 4.2846, val loss 4.2823
step 500: train loss 1.8772, val loss 1.9839
step 1000: train loss 1.5299, val loss 1.7114
step 1500: train loss 1.3905, val loss 1.6044
step 2000: train loss 1.3086, val loss 1.5457
step 2500: train loss 1.2477, val loss 1.5085
step 3000: train loss 1.2049, val loss 1.4964
step 3500: train loss 1.1607, val loss 1.4806
step 4000: train loss 1.1220, val loss 1.4766
step 4500: train loss 1.0885, val loss 1.4809

Thy women divorces with me,
Whethough gross than it is off?

AUFIDIUS:
In go alive,
Without lack more passatives; bore of my grave,
Though they do perhappy heart.
Here husband, and Bona: thou wish them side;
He may mercy coloure pleaset me, though I shall not
Choose my sacred be not a man interchange?
Nay, if I never be,
And Messfreth, one curties oft praised and to thee
Ups of my vengeance pericion.

BENVOLIO:
Go it home too longue to grief, but lips
That heart cried ways and no ward:--whom mars, do smeal
To thy g